<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">2014 South Napa Earthquake</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Prepare Waveforms</div>
        </div>
    </div>
</div>

### **2017 CIG-LLNL Computational Seismology Workshop**


##### Authors:
* Artie Rogers
* Lion Krischer ([@krischer](https://github.com/krischer))
---

4) Prepare waveforms

Basic algorithm to prepare waveforms obtained from request in previous notebook. Assumes waveforms (miniseed files) were written to directory named with event_id/RAW. This program writes SAC files with physical ground motion units and headers complete with event origin time as reference time, event and station locations, etc... 
It also rotates the horizontal components, if they both exist.

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

In [ ]:
from os import path, chdir, makedirs
from sys import exit
import obspy

Set parameters

In [ ]:
# Set eventid here
event_id = 72282711

pre_filt = (0.03, 0.04, 8.0, 10.0)
output_ground_motion = "VEL"

Check if event_id directory exists

In [ ]:
event_dir = str(event_id)

if path.exists(event_dir) == False:
    print('Directory for this event_id does not exist: ', event_dir)
    exit()
else:
    if not path.exists(event_dir+'/'+output_ground_motion):
        makedirs(event_dir+'/'+output_ground_motion)
    chdir(event_dir)

Read quakeml file to get event metadata. Read all mseed files in RAW directory.

In [ ]:
# Read QuakeML file to get event metadata
cat = obspy.read_events(event_dir+'.QUAKEML')
event = cat[0]

st = obspy.read('RAW/*.mseed')
#print(st)
#st.plot()

In [ ]:
st.plot()


Loop over all traces in stream. Remove instrument response using pre-filter window set above.

Write SAC file for each trace with header complete with event and station locations, "0" reference time as event origin time.

In [ ]:
for tr in st:
    print('net, sta, chan: ', tr.stats.network, tr.stats.station, tr.stats.channel)
    inv = obspy.read_inventory('_station_xml/'+tr.stats.network+'.'+tr.stats.station+'.xml')
    tr.remove_response(inventory=inv, pre_filt=pre_filt, output=output_ground_motion, water_level=60)
    tr.plot()
    
    seed_id = tr.stats.network+'.'+tr.stats.station+'.'+tr.stats.location+'.'+tr.stats.channel
    coords = inv.get_coordinates(seed_id)
    network_inv = inv[0]
    sta_inv = network_inv[0]
    chan_inv = sta_inv.select(channel=tr.stats.channel)
    if tr.stats.channel[-1] == 'Z':
        comp = 'z'
    if tr.stats.channel[-1] == 'N':
        comp = 'n'
    if tr.stats.channel[-1] == 'E':
        comp = 'e'
    
    sac = obspy.io.sac.sactrace.SACTrace.from_obspy_trace(tr)
    sac.stla = coords['latitude']
    sac.stlo = coords['longitude']
    sac.stel = coords['elevation']
    sac.stdp = coords['local_depth']
    sac.cmpaz = chan_inv[0].azimuth
    sac.cmpinc = 90.+chan_inv[0].dip
    sac.evla = event.origins[0].latitude
    sac.evlo = event.origins[0].longitude
    sac.evdp = event.origins[0].depth/1000
    sac.reftime = event.origins[0].time
    sac.o = 0.0
    
    epi_dist, az, baz = obspy.geodetics.gps2dist_azimuth(sac.evla, sac.evlo, sac.stla, sac.stlo)
    print('epi_dist, az, baz', epi_dist, az, baz)
    
    filename = tr.stats.network+'.'+tr.stats.station+'.'+comp+'.sac'
    sac.write(output_ground_motion+'/'+filename)


Rotate horizontals to yield radial and transverse component SAC files, if both horizontals (north & east) exist.

In [ ]:
# Create list of stations
net_sta_list = []
for tr in st:
    print('net.sta: ', tr.stats.network+'.'+tr.stats.station)
    net_sta_list.append(tr.stats.network+'.'+tr.stats.station)
net_sta_list = sorted(set(net_sta_list))
print('net_sta_list: ', net_sta_list)
# Loop over stations
for net_sta in net_sta_list:
    net = net_sta.split('.')[0]
    sta = net_sta.split('.')[1]
    if path.isfile(output_ground_motion+'/'+net+'.'+sta+'.n.sac') and path.isfile(output_ground_motion+'/'+net+'.'+sta+'.e.sac'):
        print('rotate: ', sta)
        st = obspy.read(output_ground_motion+'/'+net+'.'+sta+'.[ne].sac')
        tr_n = st[0]
        tr_e = st[1]
        sac_n = obspy.io.sac.sactrace.SACTrace.from_obspy_trace(tr_n)
        sac_e = obspy.io.sac.sactrace.SACTrace.from_obspy_trace(tr_e)
        baz = tr_n.stats.sac.baz
        st.rotate(method='NE->RT', back_azimuth=baz)
        tr_r = st[0]
        tr_t = st[1]
        sac_r = obspy.io.sac.sactrace.SACTrace.from_obspy_trace(tr_r)
        sac_t = obspy.io.sac.sactrace.SACTrace.from_obspy_trace(tr_t)
        sac_r.cmpaz = baz - 180.
        if sac_r.cmpaz < 0.:
            sac_r.cmpaz = sac_r.cmpaz + 360.
        if sac_r.cmpaz > 360.:
            sac_r.cmpaz = sac_r.cmpaz - 360.
        sac_t.cmpaz = baz - 270.
        if sac_t.cmpaz < 0.:
            sac_t.cmpaz = sac_t.cmpaz + 360.
        if sac_t.cmpaz > 360.:
            sac_t.cmpaz = sac_t.cmpaz - 360.
        sac_r.write(output_ground_motion+'/'+net+'.'+sta+'.r.sac')
        sac_t.write(output_ground_motion+'/'+net+'.'+sta+'.t.sac')
    else:
        print('Do not have two horizontals for station: ', sta)